In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

 
import warnings
warnings.filterwarnings('ignore') 
# 不发出警告

from bokeh.plotting import figure,show,output_file
output_file("line.html")
from bokeh.models import ColumnDataSource
# 导入图表绘制、图标展示模块
# 导入ColumnDataSource模块

import os
os.chdir('C:/Users/Administrator/Desktop/项目/订单拆分LTV/')


from bokeh.models import HoverTool
from bokeh.core.properties import value

In [98]:

'''
(1)数据加载及清洗
'''

#读取数据
data_order = pd.read_excel('霸王雄心11.5-11.25.xlsx',sheetname = '交易明细')
data_time = pd.read_excel('霸王雄心11.5-11.25.xlsx',sheetname = '登录时间')

#筛除NA数据
data = pd.merge(data_order,data_time,left_on = '用户名(66UIN)',right_on = '账号',how = 'left')[['交易时间','用户名(66UIN)','UCID','原价','游戏注册时间']]
data = data.dropna()

#筛除内部账号
data = data[(data['用户名(66UIN)'] != 9889)&(data['用户名(66UIN)'] != 892242)&(data['用户名(66UIN)'] != 893946)&(data['用户名(66UIN)'] != 388)&(data['用户名(66UIN)'] != 400509)]

#抽离交易时间
data.index = data['交易时间']
data['交易时间_天'] = data.index.day
data['交易时间_日'] = data['交易时间_天'].astype(str) + '日'

#渠道筛选
data = data[data['UCID'] == 217719] 

In [99]:

'''
(2)以渠道维度筛选数据
'''

data.sort_values('游戏注册时间',inplace = True)
time_lst = data['游戏注册时间'].unique()


m = []
for time in time_lst:
    data_select = data[data['游戏注册时间'] == time]
    m.append(data_select)

m[0]

,交易时间,用户名(66UIN),UCID,原价,游戏注册时间,交易时间_天,交易时间_日
交易时间,,,,,,,
2019-11-06 20:03:00,2019-11-06 20:03:00,894781,217719,1,2019-11-06,6,6日
2019-11-06 20:24:00,2019-11-06 20:24:00,894964,217719,1,2019-11-06,6,6日


In [102]:
#计算LTV

def f1(lst):
    s = []
    for df in lst:
        df_g = df.groupby('交易时间_天').sum()[['原价','UCID']].T
        s.append(df_g)
    return(s)
f1(m)

[交易时间_天       6
 原价           2
 UCID    435438, 交易时间_天      7       8       12      17
 原价          54       6      36       6
 UCID    870876  217719  435438  217719, 交易时间_天      8       9       11      19
 原价          24       6      12      30
 UCID    870876  217719  217719  217719, 交易时间_天      9       10
 原价          13       6
 UCID    653157  217719, 交易时间_天       10
 原价          142
 UCID    1741752, 交易时间_天      11
 原价           6
 UCID    217719, 交易时间_天      16
 原价           9
 UCID    870876, 交易时间_天      16       18      19      20
 原价           6     2062     660     708
 UCID    217719  2830347  653157  653157, 交易时间_天      23
 原价           6
 UCID    217719]

In [103]:
s = pd.concat([f1(m)[0],f1(m)[1],f1(m)[2],f1(m)[3],f1(m)[4],f1(m)[5],f1(m)[6],f1(m)[7],f1(m)[8]])
s = s.reset_index().fillna(0)
s = s[s['index'].isin(['原价'])]
s = s.iloc[:,1:].cumsum(axis = 1)
s.index = s.columns[:len(s)]
s

交易时间_天,6,7,8,9,10,11,12,16,17,18,19,20,23
交易时间_天,,,,,,,,,,,,,
6,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
7,0.0,54.0,60.0,60.0,60.0,60.0,96.0,96.0,102.0,102.0,102.0,102.0,102.0
8,0.0,0.0,24.0,30.0,30.0,42.0,42.0,42.0,42.0,42.0,72.0,72.0,72.0
9,0.0,0.0,0.0,13.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0
10,0.0,0.0,0.0,0.0,142.0,142.0,142.0,142.0,142.0,142.0,142.0,142.0,142.0
11,0.0,0.0,0.0,0.0,0.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,9.0,9.0,9.0,9.0,9.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,6.0,2068.0,2728.0,3436.0,3436.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0


9